In [1]:
import copy

# 外皮情報LV2からLV3へのコンバート

## 1. 外皮性能計算プログラム＋簡単な追加情報

- 外皮性能計算プログラムの入力情報に、用途別床面積を追加したものより、推定を行う。
- 現状の外皮性能計算プログラムで入力されるのは用途別に分かれていない外皮性能であるため、室用途別の性能を求めるためには、外皮を用途別に割り振る必要がある。
- 土間床等面積が0㎡より大きい場合、基礎断熱部は床下空間に属するものとみなし、室と床下空間の間の床の合計面積は土間床等面積の合計に等しいとみなす。室と床下空間の間の床は、仕様を想定して与える（合板12mm、参考：SimHeatでの熱負荷計算用の無断熱の層構成）
- 住戸全体の外皮（床下空間に属するものを除く）、および、室と床下空間の間の床は、室用途別の床面積に応じて各用途に割り当てる。集合住宅の戸境壁等も同様の扱いとする。外皮性能は用途によらず均一に割り付ける。開口部は、外皮性能計算プログラムの入力情報より推定を開始する場合は、庇等の日射遮蔽部材を考慮した計算を行うため、方位、遮蔽部材やその寸法等が異なる場合は、別部材として扱う。
- 各部位の熱橋は、対応する一般部のU値の低減に置き換える。一般部の断熱材の厚を調整する。
- その他は、用途別情報が得られている場合と同様に求める。

### 《土間床等面積が0㎡より大きい場合の扱い》
$$ \qquad
A_{b}=min(A_主 + A_他 + A_非, \sum_{i}^{}S_{df,i})
\qquad
\\
$$

- 入力値：
  - 室用途不明の空間に属する土間床等の部位iの面積$[m ^ 2]$：$S_{df,i}$

- 出力値：

  - 室と床下空間の間の床面積の合計$[m ^ 2]$：$A_{b}$

### 1) 室用途別面積比率

#### Outline

$$ 
r_m = \frac{ a_m }{ A_m + A_o + A_n }
$$

$$
r_o = \frac{ A_o }{ A_m + A_o + A_n }
$$

$$
r_n = \frac{ A_n }{ A_m + A_o + A_n }
$$

$r_m$: 主たる居室の面積比率, -  
$r_o$: その他の居室の面積比率, -  
$r_n$: 非居室の面積比率, -  
$A_m$: 主たる居室の面積, m<sup>2</sup>  
$A_o$: その他の居室の面積, m<sup>2</sup>  
$A_n$: 非居室の面積, m<sup>2</sup>  

#### Function

In [2]:
def get_area_rate(a_m, a_o, a_t):
    # a_m is the main occupied floor area as float, m2
    # a_o is the other occupied floor area as float, m2
    # a_t is the total occupied floor area as float, m2
    
    # a_n is the non ocupied floor area as float, m2
    a_n = a_t - a_m - a_o

    return a_m / a_t, a_o / a_t, a_n / a_t

### 2) 室用途別の面積の推定

#### Outline

$$
A_{m,i}= A_{i} \times r_m
$$

$$
A_{o,i}= A_{i} \times r_o
$$

$$
A_{n,i}= A_{i} \times r_n
$$

$A_{m,i}$: 主たる居室の部位(一般部位又は開口部)$i$の面積, m<sup>2</sup>  
$A_{o,i}$: その他の居室の部位(一般部位又は開口部)$i$の面積, m<sup>2</sup>  
$A_{n,i}$: 非居室の部位(一般部位又は開口部)$i$の面積, m<sup>2</sup>  
$A_i$: 外皮の部位(一般部位又は開口部)$i$の面積, m<sup>2</sup>

#### Function

In [3]:
def get_spaceareas( target_area, a_m, a_o, a_t ):
    # target_area is the area of the part i as double, m2
    # a_m is the main floor area as float, m2
    # a_o is the other floor area as float, m2
    # a_t is the total floor area as float, m2
    
    r_m, r_o, r_n = get_area_rate( a_m, a_o, a_t )
    return {
        'main'      : r_m * target_area,
        'other'     : r_o * target_area,
        'nonliving' : r_n * target_area
    }

### 3) 室用途別の長さの推定

#### Outline

$$
L_{m,i}= L_{i} \times r_m
$$

$$
L_{o,i}= L_{i} \times r_o
$$

$$
L_{n,i}= L_{i} \times r_n
$$

$L_{m,i}$: 主たる居室の熱橋及び土間床等の外周部$i$の長さ, m  
$L_{o,i}$: その他の熱橋及び土間床等の外周部$i$の長さ, m  
$L_{n,i}$: 非居室の熱橋及び土間床等の外周部$i$の長さ, m  
$L_i$: 熱橋及び土間床等の外周部$i$の長さ, m

#### Function

In [4]:
def get_spacelengths(target_length, a_m, a_o, a_t):
    # target_length is the length of the part i as double, m
    # a_m is the main floor area as float, m
    # a_o is the other floor area as float, m
    # a_t is the total floor area as float, m
    
    r_m, r_o, r_n = get_area_rate(a_m, a_o, a_t)
    return {
        'main'      : r_m * target_length,
        'other'     : r_o * target_length,
        'nonliving' : r_n * target_length
    }

#### Example

In [5]:
areas = get_spaceareas(50.0, 30.0, 60.0, 120.0)
lengths = get_spacelengths(12.0, 30.0, 60.0, 120.0)
areas, lengths

({'main': 12.5, 'other': 25.0, 'nonliving': 12.5},
 {'main': 3.0, 'other': 6.0, 'nonliving': 3.0})

## 2. Convert

### 1) 共通

'Common'要素はそのまま維持する。

#### Input

参照：InputDataDefinition.ipynb

#### Output

参照：InputDataDefinition.ipynb

#### Function

In [6]:
def convert_common(d):
    return d['Common']

#### Example

In [7]:
d = {
    'Common': {
        'Region': 6,
        'MainOccupantRoomFloorArea': 30.0,
        'OtherOccupantRoomFloorArea': 60.0,
        'TotalFloorArea': 120.0
    }
}

convert_common(d)

{'Region': 6,
 'MainOccupantRoomFloorArea': 30.0,
 'OtherOccupantRoomFloorArea': 60.0,
 'TotalFloorArea': 120.0}

### 2) 開口部以外の不透明部位

- 壁体名称、一般部位構造種別、壁体種別、方位、層構成（部材名称、厚さ[m]、熱伝導率[W/mK]、容積比熱[(J/(m<sup>3</sup>・K)]）などの情報は維持する。
- Lv2の入力情報に「壁体が属する空間種類」の情報を追加し、部位面積から各部位の室用途別面積を推定する。

#### Function

In [8]:
def make_layer(name, thick, cond, specH, **kwargs) :
    
    return {
            'name'  : name,
            'thick' : thick,
            'cond'  : cond,
            'specH' : specH
    }

In [9]:
d_part_type = {
    'Roof':{
        'RoofInsulRafter':['RoofInsulation','RoofHeatbridge'],
        'RoofInsulRafterandadd':['RoofInsulRafterandadd','RoofInsulrafterHeatbridgeadd',
                                 'RoofInsuladdHeatbridgemember','RoofHeatbridgeMemberandadd'],
        'OuterInsul':['RoofInsulRafterandadd','RoofInsulrafterHeatbridgeadd','RoofInsuladdHeatbridgemember','RoofHeatbridgeMemberandadd']
    },
    'Wall':{
        'WallFrameInsulcolumn':['WallInsulation','WallHeatbridge'],
        'WallFrameInsuladdBackhorizontal':['WallFrameInsulColumnandadd','WallFrameInsulcolumnHeatbridgeadd',
                                           'WallFrameInsuladdHeatbridgemember','WallFrameHeatbridgeMemberandadd'],
        'WallFrameInsuladdBackvertical':['WallFrameInsulColumnandadd','WallFrameInsulcolumnHeatbridgeadd',
                                         'WallFrameInsuladdHeatbridgemember','WallFrameHeatbridgeMemberandadd'],
        'WallInsuljamb':['WallInsulation','WallHeatbridge'],
        'WallInsuladdBackhorizontal':['WallInsulFilledandadd','WallInsulfilledHeatbridgeadd','WallInsuladdHeatbridgemember',
                                      'WallInsuladdHeatbridgelintel','WallHeatbridgeMemberandadd','WallHeatbridgeLintelandadd'],
        'WallInsuladdBackvertical':['WallInsulFilledandadd','WallInsulfilledHeatbridgeadd','WallInsuladdHeatbridgemember',
                                    'WallInsuladdHeatbridgelintel','WallHeatbridgeMemberandadd','WallHeatbridgeLintelandadd']
    },
    'BoundaryWall':{
        'WallFrameInsulcolumn':['WallInsulation','WallHeatbridge'],
        'WallFrameInsuladdBackhorizontal':['WallFrameInsulColumnandadd','WallFrameInsulcolumnHeatbridgeadd',
                                           'WallFrameInsuladdHeatbridgemember','WallFrameHeatbridgeMemberandadd'],
        'WallFrameInsuladdBackvertical':['WallFrameInsulColumnandadd','WallFrameInsulcolumnHeatbridgeadd',
                                         'WallFrameInsuladdHeatbridgemember','WallFrameHeatbridgeMemberandadd'],
        'WallInsuljamb':['WallInsulation','WallHeatbridge'],
        'WallInsuladdBackhorizontal':['WallInsulFilledandadd','WallInsulfilledHeatbridgeadd','WallInsuladdHeatbridgemember',
                                      'WallInsuladdHeatbridgelintel','WallHeatbridgeMemberandadd','WallHeatbridgeLintelandadd'],
        'WallInsuladdBackvertical':['WallInsulFilledandadd','WallInsulfilledHeatbridgeadd','WallInsuladdHeatbridgemember',
                                    'WallInsuladdHeatbridgelintel','WallHeatbridgeMemberandadd','WallHeatbridgeLintelandadd']
    },
    'Floor':{
        'FloorFrameBeamInsuljoist':['FloorInsulation','FloorHeatbridge'],
        'FloorFrameFootingInsuljoist':['FloorInsulation','FloorHeatbridge'],
        'FloorFrameFootingInsulsleeper':['FloorInsulation','FloorHeatbridge'],
        'FloorFrameFootingInsuljoist_sleeper':['FloorInsulJoitandsleeper','FloorInsuljoistHeatbridgesleeper',
                                               'FloorInsulsleeperHeatbridgejoist','FloorHeatbridgeJoistandsleeper'],
        'FloorFrameRigidfloor':['FloorInsulation','FloorHeatbridge'],
        'FloorFrameBeambaseInsuljoist':['FloorInsulation','FloorHeatbridge'],
        'FloorWallInsuljoist':['FloorInsulation','FloorHeatbridge']
    },
    'BoundaryFloor':{
        'FloorFrameBeamInsuljoist':['FloorInsulation','FloorHeatbridge'],
        'FloorFrameFootingInsuljoist':['FloorInsulation','FloorHeatbridge'],
        'FloorFrameFootingInsulsleeper':['FloorInsulation','FloorHeatbridge'],
        'FloorFrameFootingInsuljoist_sleeper':['FloorInsulJoitandsleeper','FloorInsuljoistHeatbridgesleeper',
                                               'FloorInsulsleeperHeatbridgejoist','FloorHeatbridgeJoistandsleeper'],
        'FloorFrameRigidfloor':['FloorInsulation','FloorHeatbridge'],
        'FloorFrameBeambaseInsuljoist':['FloorInsulation','FloorHeatbridge'],
        'FloorWallInsuljoist':['FloorInsulation','FloorHeatbridge']
    },
    'Ceiling':{
        'CeilingInsulbeam':['CeilingInsulation','CeilingHeatbridge']
    },
    'BoundaryCeiling':{
        'CeilingInsulbeam':['CeilingInsulation','CeilingHeatbridge']
    }    
}

def make_parts(InputMethod, Parts, type = None, ConstructionMethodAllLayers = None, **kwargs) :
    
    parts_copy = []
    
    for x in Parts :
        part = {}
        if InputMethod == 'InputAllDetails' :
            part['AreaRatio'] = x['AreaRatio']
        elif InputMethod == 'InputAllLayers' :
            part['Type'] = x['Type']
            if (type in d_part_type) == True :
                if (ConstructionMethodAllLayers in d_part_type[type]) == True :
                    if (part['Type'] in d_part_type[type][ConstructionMethodAllLayers]) == True :
                        pass
                    else :
                        raise ValueError
                else :
                    raise ValueError
            else :
                raise ValueError
            
        part['Layers'] = [make_layer(**layer) for layer in x['Layers']]
        parts_copy.append(part)
        
    return parts_copy

In [10]:
def make_wall(name, nextspace, direction, spacename, spacearea, type, InputMethod, IsSunshadeInput, 
              UA = None, structure = None, IsInContactWithOutsideAir = None, UR = None,
              ConstructionMethodAllLayers = None, ConstructionMethodUR = None, InsulationMethod = None, 
              Radding = None, Parts = None, Y1 = None, Y2 = None, Z = None, **kwargs ):
    
    wall = {
        'name':      name + '_' + spacename,
        'nextspace': nextspace,
        'direction': direction,
        'area':      spacearea,
        'space':     spacename,
        'type':      type,
        'InputMethod': InputMethod,
        'IsSunshadeInput': IsSunshadeInput        
    }
    
    if InputMethod == 'InputUA' :
            wall['UA'] = UA 
    else :
        wall['structure'] = structure
        if structure == 'wood' :
            if InputMethod == 'InputAllDetails' :
                wall['IsInContactWithOutsideAir'] = IsInContactWithOutsideAir
                wall['Parts'] = make_parts(InputMethod = InputMethod, Parts = Parts)
            elif InputMethod == 'InputAllLayers' :
                wall['IsInContactWithOutsideAir'] = IsInContactWithOutsideAir
                wall['Parts'] = make_parts(InputMethod = InputMethod, ConstructionMethodAllLayers = ConstructionMethodAllLayers,
                                           Parts = Parts, type = type)
                wall['ConstructionMethodAllLayers'] = ConstructionMethodAllLayers           
            elif InputMethod == 'InputUR' :
                wall['IsInContactWithOutsideAir'] = IsInContactWithOutsideAir
                if type == 'Floor' or type == 'BoundaryCeiling' or type == 'BoundaryFloor' :
                    wall['ConstructionMethodUR'] = ConstructionMethodUR
                elif type == 'Wall' or type == 'BoundaryWall' :
                    wall['InsulationMethod'] = InsulationMethod
                    if WallInsulationMethod == 'InsulColumn' or WallInsulationMethod == 'InsulColumnandadd' :
                        wall['ConstructionMethodUR'] = ConstructionMethodUR       
                elif type == 'Ceiling' :
                    wall['InsulationMethod'] = InsulationMethod
                elif type == 'Roof' :    
                    wall['InsulationMethod'] = InsulationMethod                    
                else :
                    raise ValueError 
                wall['Parts'] = make_parts(InputMethod = InputMethod, Parts = Parts)
            else :
                raise ValueError                    
        elif structure == 'RC' :
            if InputMethod == 'InputLayers' :
                wall['IsInContactWithOutsideAir'] = IsInContactWithOutsideAir
                wall['Parts'] = make_parts(InputMethod = InputMethod, Parts = Parts)
            else :
                raise ValueError            
        elif structure == 'steel' :
            if InputMethod == 'InputUR' :
                wall['Radding'] = Radding
                wall['IsInContactWithOutsideAir'] = IsInContactWithOutsideAir
                wall['Parts'] = make_parts(InputMethod = InputMethod, Parts = Parts)
            else :
                raise ValueError        
        elif structure == 'other' :
            pass
        else :
            raise ValueError        

    # 日除けの設定
    if IsSunshadeInput == True :
        wall['Y1'] = Y1
        wall['Y2'] = Y2
        wall['Z'] = Z
 
    return wall

In [11]:
def convert_wall( d ):
    walls =[]
    for arg_wall in d['Walls']:
        spaceareas = get_spaceareas(
            target_area=arg_wall['area'], 
            a_m = d['Common']['MainOccupantRoomFloorArea'], 
            a_o = d['Common']['OtherOccupantRoomFloorArea'], 
            a_t = d['Common']['TotalFloorArea']
        )
        for spacename, spacearea in spaceareas.items() :
             # 壁体名称、部位面積を更新し、「壁体が属する空間種類」を追加する
            wall = make_wall(
                **arg_wall,
                spacename = spacename,
                spacearea = spacearea
            )
            walls.append(wall)
            
    return walls

#### Example

In [12]:
d = {
    'Common': {
        'Region': 6,
        'MainOccupantRoomFloorArea': 30.0,
        'OtherOccupantRoomFloorArea': 60.0,
        'TotalFloorArea': 120.0
    },
    'Walls': [
        { 'name': 'Ceiling', 'type': 'Ceiling', 'structure': 'wood', 'InputMethod': 'InputUA', 
          'nextspace': 'Outdoor', 'direction': 'top', 'area': 67.8, 'UA': 0.24, 'IsSunshadeInput': False },
        { 'name': 'Ceiling', 'type': 'Ceiling', 'structure': 'wood', 'InputMethod': 'InputAllDetails', 
          'nextspace': 'Outdoor', 'direction': 'top', 'IsInContactWithOutsideAir': True, 'area': 67.8,
          'Parts': [{'AreaRatio': 0.8, 'Layers': [{'name': 'wood','thick': 0.012, 'cond': 0.16, 'specH': 720 },
                                                  {'name': 'wood','thick': 0.012, 'cond': 0.16, 'specH': 720 }]},
                    {'AreaRatio': 0.2, 'Layers': [{'name': 'wood','thick': 0.012, 'cond': 0.16, 'specH': 720 },
                                                  {'name': 'wood','thick': 0.012, 'cond': 0.16, 'specH': 720 }]}],
          'IsSunshadeInput': False },
        { 'name': 'Floor', 'type': 'Floor', 'structure': 'wood', 'InputMethod': 'InputAllLayers','nextspace': 'Outdoor',  'direction': 'bottom', 
          'IsInContactWithOutsideAir': True, 'area': 67.8,
          'ConstructionMethodAllLayers' :'FloorFrameBeamInsuljoist',
          'Parts': [{'Type': 'FloorInsulation', 'Layers': [{'name': 'wood','thick': 0.012, 'cond': 0.16, 'specH': 720 },
                                                           {'name': 'wood','thick': 0.012, 'cond': 0.16, 'specH': 720 }]},
                    {'Type': 'FloorHeatbridge', 'Layers': [{'name': 'wood','thick': 0.012, 'cond': 0.16, 'specH': 720 },
                                                           {'name': 'wood','thick': 0.012, 'cond': 0.16, 'specH': 720 }]}],
           'IsSunshadeInput': False },
        { 'name': 'Wall', 'type': 'Wall', 'structure': 'wood', 'InputMethod': 'InputAllLayers', 'nextspace': 'Outdoor', 'direction': 'N',
          'IsInContactWithOutsideAir': True, 'area': 67.8, 'ConstructionMethodAllLayers': 'WallFrameInsulcolumn',
          'Parts': [{'Type': 'WallInsulation', 'Layers': [{'name': 'wood','thick': 0.012, 'cond': 0.16, 'specH': 720 },
                                                          {'name': 'wood','thick': 0.012, 'cond': 0.16, 'specH': 720 }]},
                    {'Type': 'WallHeatbridge', 'Layers': [{'name': 'wood','thick': 0.012, 'cond': 0.16, 'specH': 720 },
                                                          {'name': 'wood','thick': 0.012, 'cond': 0.16, 'specH': 720 }]}],         
          'IsSunshadeInput': False },
        { 'name': 'Ceiling', 'type': 'Roof', 'structure': 'wood', 'InputMethod': 'InputAllLayers', 'nextspace': 'Outdoor', 'direction': 'top',
          'IsInContactWithOutsideAir': True, 'area': 67.8, 'ConstructionMethodAllLayers': 'RoofInsulRafter', 
          'Parts': [{'Type': 'RoofInsulation', 'Layers': [{'name': 'wood','thick': 0.012, 'cond': 0.16, 'specH': 720 },
                                                          {'name': 'wood','thick': 0.012, 'cond': 0.16, 'specH': 720 }]},
                    {'Type': 'RoofHeatbridge', 'Layers': [{'name': 'wood','thick': 0.012, 'cond': 0.16, 'specH': 720 },
                                                          {'name': 'wood','thick': 0.012, 'cond': 0.16, 'specH': 720 }]}],
          'IsSunshadeInput': False },
        { 'name': 'BoundaryCeiling', 'type': 'BoundaryCeiling', 'structure': 'wood', 'InputMethod': 'InputAllLayers','nextspace': 'Outdoor',  
          'direction': 'top', 'IsInContactWithOutsideAir': True, 'area': 67.8, 'ConstructionMethodAllLayers': 'CeilingInsulbeam', 
          'Parts': [{'Type': 'CeilingInsulation', 'Layers': [{'name': 'wood','thick': 0.012, 'cond': 0.16, 'specH': 720 },
                                                             {'name': 'wood','thick': 0.012, 'cond': 0.16, 'specH': 720 }]},
                    {'Type': 'CeilingHeatbridge', 'Layers': [{'name': 'wood','thick': 0.012, 'cond': 0.16, 'specH': 720 },
                                                             {'name': 'wood','thick': 0.012, 'cond': 0.16, 'specH': 720 }]}],
          'IsSunshadeInput': False },
        { 'name': 'Ceiling', 'type': 'Ceiling', 'structure': 'wood', 'InputMethod': 'InputUR', 'InsulationMethod': 'CeilingInsulbeam',
          'nextspace': 'Outdoor', 'direction': 'top', 'IsInContactWithOutsideAir': True, 'area': 67.8,
          'Parts': [{ 'Layers': [{'name': 'wood','thick': 0.012, 'cond': 0.16, 'specH': 720 },
                                 {'name': 'wood','thick': 0.012, 'cond': 0.16, 'specH': 720 }]}],
          'IsSunshadeInput': False },
        { 'name': 'Ceiling', 'type': 'Ceiling', 'structure': 'RC', 'InputMethod': 'InputUA', 
          'nextspace': 'Outdoor', 'direction': 'top', 'area': 67.8, 'UA': 0.24, 'IsSunshadeInput': False },
        { 'name': 'Ceiling', 'type': 'Ceiling', 'structure': 'RC', 'InputMethod' :'InputLayers', 
          'nextspace': 'Outdoor', 'direction': 'top', 'area': 67.8,
          'Parts': [{'Layers': [{'name': 'wood','thick': 0.012, 'cond': 0.16, 'specH': 720 },
                                {'name': 'wood','thick': 0.012, 'cond': 0.16, 'specH': 720 }]}],         
          'IsSunshadeInput': False },
        { 'name': 'Ceiling', 'type': 'Ceiling', 'structure': 'steel', 'InputMethod': 'InputUA', 
          'nextspace': 'Outdoor', 'direction': 'top', 'area': 67.8, 'UA': 0.24, 'IsSunshadeInput': False },
        { 'name': 'Ceiling', 'type': 'Ceiling', 'structure': 'steel', 'InputMethod' :'InputUR', 
          'nextspace': 'Outdoor', 'direction': 'top', 'IsInContactWithOutsideAir': True, 'area': 67.8, 'Radding': 0.10, 
          'Parts': [{'Layers': [{'name': 'wood','thick': 0.012, 'cond': 0.16, 'specH': 720 },
                                {'name': 'wood','thick': 0.012, 'cond': 0.16, 'specH': 720 }]}],         
          'IsSunshadeInput': False },
        { 'name': 'Ceiling', 'type': 'Ceiling', 'structure': 'other', 'InputMethod': None, 'nextspace': 'Outdoor', 
          'direction': 'top', 'area': 67.8, 'IsSunshadeInput': False }
    ]
} 

convert_wall(d)

[{'name': 'Ceiling_main',
  'nextspace': 'Outdoor',
  'direction': 'top',
  'area': 16.95,
  'space': 'main',
  'type': 'Ceiling',
  'InputMethod': 'InputUA',
  'IsSunshadeInput': False,
  'UA': 0.24},
 {'name': 'Ceiling_other',
  'nextspace': 'Outdoor',
  'direction': 'top',
  'area': 33.9,
  'space': 'other',
  'type': 'Ceiling',
  'InputMethod': 'InputUA',
  'IsSunshadeInput': False,
  'UA': 0.24},
 {'name': 'Ceiling_nonliving',
  'nextspace': 'Outdoor',
  'direction': 'top',
  'area': 16.95,
  'space': 'nonliving',
  'type': 'Ceiling',
  'InputMethod': 'InputUA',
  'IsSunshadeInput': False,
  'UA': 0.24},
 {'name': 'Ceiling_main',
  'nextspace': 'Outdoor',
  'direction': 'top',
  'area': 16.95,
  'space': 'main',
  'type': 'Ceiling',
  'InputMethod': 'InputAllDetails',
  'IsSunshadeInput': False,
  'structure': 'wood',
  'IsInContactWithOutsideAir': True,
  'Parts': [{'AreaRatio': 0.8,
    'Layers': [{'name': 'wood', 'thick': 0.012, 'cond': 0.16, 'specH': 720},
     {'name': 'wood'

### 3) 開口部の透明部位

- 開口部名称、方位、日射熱取得率[-]、熱貫流率[W/(m<sup>2</sup>・K)]などの情報は維持する。
- 「開口部が属する空間種類」の情報を追加し、部位面積から各部位の室用途別面積を推定する。

#### Function

In [13]:
def make_window(name, nextspace, direction, spacename, spacearea, IsSunshadeInput, EtaInputMethod, UW, TypeWindow = None, 
                EtaCooling = None, EtaHeating = None, Eta = None,TypeFrame = None, TypeGlass = None,  
                TypeShade = None, EtaInside = None,TypeFrameInside = None, TypeGlassInside = None, TypeShadeInside = None, 
                EtaOutside = None,TypeFrameOutside = None, TypeGlassOutside = None, TypeShadeOutside = None, 
                Y1 = None, Y2 = None, Z = None, **kwargs ):
    
    window = {
        'name'      : name + '_' + spacename,
        'nextspace' : nextspace,
        'direction' : direction,
        'area'      : spacearea,
        'space'     : spacename,
        'UW'        : UW,
        'TypeWindow'     : TypeWindow,
        'EtaInputMethod' : EtaInputMethod,
        'IsSunshadeInput': IsSunshadeInput        
    } 
    
    # 数値の入力
    if EtaInputMethod == 'InputValue' :
        # 一重窓
        if TypeWindow == 'Single' :
            window['Eta'] = Eta
        # 二重窓
        elif TypeWindow == 'Double' :
            window['EtaInside'] = EtaInside
            window['EtaOutside'] = EtaOutside
        else :
            raise ValueError   
    # 仕様の入力
    elif EtaInputMethod == 'InputSpecification' :
        # 一重窓
        if TypeWindow == 'Single' :
            window['TypeFrame'] = TypeFrame
            window['TypeGlass'] = TypeGlass
            window['TypeShade'] = TypeShade
        # 二重窓
        elif TypeWindow == 'Double' :
            window['TypeFrameInside'] = TypeFrameInside
            window['TypeGlassInside'] = TypeGlassInside
            window['TypeShadeInside'] = TypeShadeInside
            window['TypeFrameOutside'] = TypeFrameOutside
            window['TypeGlassOutside'] = TypeGlassOutside
            window['TypeShadeOutside'] = TypeShadeOutside
        else :
            raise ValueError        
    else :
        raise ValueError  
    
    # 日除けの設定
    if IsSunshadeInput == True :
        # 一重窓
        if TypeWindow == 'Single' :
                window['TypeGlass'] = TypeGlass
        # 二重窓
        elif TypeWindow == 'Double' :
                window['TypeGlassInside'] = TypeGlassInside
                window['TypeGlassOutside'] = TypeGlassOutside
        else :
            raise ValueError        
                
        window['Y1'] = Y1
        window['Y2'] = Y2
        window['Z'] = Z

    return window

In [14]:
def convert_window( d ):
    
    windows =[]
    for arg_window in d['Windows']:
        areas = get_spaceareas(
            target_area = arg_window['area'], 
            a_m = d['Common']['MainOccupantRoomFloorArea'], 
            a_o = d['Common']['OtherOccupantRoomFloorArea'], 
            a_t = d['Common']['TotalFloorArea']
        )
        for spacename, spacearea in areas.items() :
            # 開口部名称、部位面積を更新し、「開口部が属する空間種類」を追加する
            window = make_window(
                **arg_window,
                spacename = spacename,
                spacearea = spacearea
            )
            windows.append(window)
        
    return windows

#### Example

In [15]:
d = {
    'Common': {
        'Region': 6,
        'MainOccupantRoomFloorArea': 30.0,
        'OtherOccupantRoomFloorArea': 60.0,
        'TotalFloorArea': 120.0
    },
    'Windows': [
        { 'name': 'WindowSW', 'nextspace': 'Outdoor', 'direction': 'SW', 'area': 30.25, 'UW': 6.51, 'TypeWindow': 'Single', 
          'EtaInputMethod': 'InputSpecification', 'TypeFrame': 'WoodOrResin', 'TypeGlass': '3WgG', 'TypeShade': 'Shoji',
          'IsSunshadeInput': True, 'Y1': 0.00, 'Y2': 1.00, 'Z': 0.60 },
        { 'name': 'WindowNW', 'nextspace': 'Outdoor', 'direction': 'NW', 'area': 3.17, 'UW': 4.65, 'TypeWindow': 'Single', 
          'EtaInputMethod': 'InputValue', 'Eta': 0.738, 'IsSunshadeInput': False },
        { 'name': 'WindowSW', 'nextspace': 'Outdoor', 'direction': 'SW', 'area': 30.25, 'UW': 6.51, 'TypeWindow': 'Double',
          'EtaInputMethod': 'InputValue', 'TypeFrameInside': 'WoodOrResin', 'TypeGlassInside': '3WgG', 'TypeShadeInside': 'Shoji',
          'TypeFrameOutside': 'Steel', 'TypeGlassOutside': '3WgG', 'TypeShadeOutside': 'ExtarnalBlind',
          'IsSunshadeInput': True, 'Y1': 0.00, 'Y2': 1.00, 'Z': 0.60 },
        { 'name': 'WindowNW', 'nextspace': 'Outdoor', 'direction': 'NW', 'area': 3.17, 'UW': 4.65, 
          'TypeWindow': 'Double', 'EtaInputMethod': 'InputValue',
          'EtaInside': 0.738, 'TypeGlassInside': '3WgG', 'EtaOutside': 0.738, 'TypeGlassOutside': '3WgG',
          'IsSunshadeInput': True, 'Y1': 0.00, 'Y2': 1.00, 'Z': 0.60 }
    ]
}    

convert_window(d)

[{'name': 'WindowSW_main',
  'nextspace': 'Outdoor',
  'direction': 'SW',
  'area': 7.5625,
  'space': 'main',
  'UW': 6.51,
  'TypeWindow': 'Single',
  'EtaInputMethod': 'InputSpecification',
  'IsSunshadeInput': True,
  'TypeFrame': 'WoodOrResin',
  'TypeGlass': '3WgG',
  'TypeShade': 'Shoji',
  'Y1': 0.0,
  'Y2': 1.0,
  'Z': 0.6},
 {'name': 'WindowSW_other',
  'nextspace': 'Outdoor',
  'direction': 'SW',
  'area': 15.125,
  'space': 'other',
  'UW': 6.51,
  'TypeWindow': 'Single',
  'EtaInputMethod': 'InputSpecification',
  'IsSunshadeInput': True,
  'TypeFrame': 'WoodOrResin',
  'TypeGlass': '3WgG',
  'TypeShade': 'Shoji',
  'Y1': 0.0,
  'Y2': 1.0,
  'Z': 0.6},
 {'name': 'WindowSW_nonliving',
  'nextspace': 'Outdoor',
  'direction': 'SW',
  'area': 7.5625,
  'space': 'nonliving',
  'UW': 6.51,
  'TypeWindow': 'Single',
  'EtaInputMethod': 'InputSpecification',
  'IsSunshadeInput': True,
  'TypeFrame': 'WoodOrResin',
  'TypeGlass': '3WgG',
  'TypeShade': 'Shoji',
  'Y1': 0.0,
  'Y2'

### 4) 開口部の不透明部位

- 開口部名称、方位、熱貫流率[W/(m<sup>2</sup>・K)]などの情報は維持する。
- 「開口部が属する空間種類」の情報を追加し、部位面積から各部位の室用途別面積を推定する。

#### Function

In [16]:
def make_door(name, nextspace, direction, spacename, spacearea, U, IsSunshadeInput, Y1 = None, Y2 = None, Z = None, **kwargs ):

    door = {
        'name':      name + '_' + spacename,
        'nextspace': nextspace,
        'direction': direction,
        'area':      spacearea,
        'space':     spacename,
        'U':         U,
        'IsSunshadeInput': IsSunshadeInput
    }
    
    # 日除けの設定
    if IsSunshadeInput == True :
        door['Y1'] = Y1
        door['Y2'] = Y2
        door['Z'] = Z
    
    return door

In [17]:
def convert_door( d ):
    
    doors =[]
    for arg_door in d['Doors']:
        spaceareas = get_spaceareas(arg_door['area'], d['Common']['MainOccupantRoomFloorArea'], d['Common']['OtherOccupantRoomFloorArea'], d['Common']['TotalFloorArea'])
        for spacename, spacearea in spaceareas.items() :
            # 開口部名称、部位面積を更新し、「開口部が属する空間種類」を追加する
            door = make_door(
                 **arg_door,
                 spacename = spacename,
                 spacearea = spacearea
             )
            doors.append(door)
        
    return doors

#### Example

In [18]:
d = {
    'Common': {
        'Region': 6,
        'MainOccupantRoomFloorArea': 30.0,
        'OtherOccupantRoomFloorArea': 60.0,
        'TotalFloorArea': 120.0
    },
    'Doors': [
        { 'name': 'DoorNW', 'nextspace': 'Outdoor', 'direction': 'NW', 'area': 2.52, 'U': 6.51, 'IsSunshadeInput': False },
        { 'name': 'DoorNE', 'nextspace': 'Outdoor', 'direction': 'NE', 'area': 2.16, 'U': 4.65, 
          'IsSunshadeInput': True, 'Y1': 0.00, 'Y2': 1.00, 'Z': 0.60 }
    ]
}    

convert_door(d)

[{'name': 'DoorNW_main',
  'nextspace': 'Outdoor',
  'direction': 'NW',
  'area': 0.63,
  'space': 'main',
  'U': 6.51,
  'IsSunshadeInput': False},
 {'name': 'DoorNW_other',
  'nextspace': 'Outdoor',
  'direction': 'NW',
  'area': 1.26,
  'space': 'other',
  'U': 6.51,
  'IsSunshadeInput': False},
 {'name': 'DoorNW_nonliving',
  'nextspace': 'Outdoor',
  'direction': 'NW',
  'area': 0.63,
  'space': 'nonliving',
  'U': 6.51,
  'IsSunshadeInput': False},
 {'name': 'DoorNE_main',
  'nextspace': 'Outdoor',
  'direction': 'NE',
  'area': 0.54,
  'space': 'main',
  'U': 4.65,
  'IsSunshadeInput': True,
  'Y1': 0.0,
  'Y2': 1.0,
  'Z': 0.6},
 {'name': 'DoorNE_other',
  'nextspace': 'Outdoor',
  'direction': 'NE',
  'area': 1.08,
  'space': 'other',
  'U': 4.65,
  'IsSunshadeInput': True,
  'Y1': 0.0,
  'Y2': 1.0,
  'Z': 0.6},
 {'name': 'DoorNE_nonliving',
  'nextspace': 'Outdoor',
  'direction': 'NE',
  'area': 0.54,
  'space': 'nonliving',
  'U': 4.65,
  'IsSunshadeInput': True,
  'Y1': 0.

### 5) 線熱橋

- 熱橋部の構造種別、方位、線熱貫流率[W/(m・K)]などの情報は維持する。
- 「 熱橋部が属する空間種類」の情報を追加し、熱橋長さから室用途別の熱橋長さを推定する。

#### Function

In [19]:
def make_heatbridge(structure, nextspace1, nextspace2, spacename, spacelength, direction1 = None, direction2 = None,  **kwargs ):
    
    heatbridge = {
        'nextspace1': nextspace1,
        'direction1': direction1,
        'nextspace2': nextspace2,
        'direction2': direction2,
        'structure':  structure,
        'length':     spacelength,
        'space':      spacename
    }

    return heatbridge

In [20]:
def convert_heatbridge( d ):
    
    heatbridges =[]
    for arg_heatbridge in d['Heatbridges']:
        lengths = get_spacelengths(arg_heatbridge['length'], d['Common']['MainOccupantRoomFloorArea'], d['Common']['OtherOccupantRoomFloorArea'], d['Common']['TotalFloorArea'])
        for spacename, spacelength in lengths.items() :
            # 熱橋部長さを更新し、「熱橋部が属する空間種類」を追加する
            heatbridge = make_heatbridge(
                **arg_heatbridge,
                spacename = spacename,
                spacelength = spacelength
            )
            heatbridges.append(heatbridge)
        
    return heatbridges

#### Example

In [21]:
d = {
    'Common': {
        'Region': 6,
        'MainOccupantRoomFloorArea': 30.0,
        'OtherOccupantRoomFloorArea': 60.0,
        'TotalFloorArea': 120.0
    },    
    'Heatbridges': [
        { 'name': 'NE', 'structure': 'RC', 'length': 1.00, 'psi': 1.8, 
          'nextspace1': 'Outdoor', 'direction1': 'N', 'nextspace2': 'Outdoor', 'direction2': 'E' },
        { 'name': 'NW', 'structure': 'Steel', 'length': 2.00, 'psi': 1.8,
          'nextspace1': 'Outdoor', 'direction1': 'N', 'nextspace2': 'Outdoor', 'direction2': 'W' }
    ]
}    

convert_heatbridge(d)

[{'nextspace1': 'Outdoor',
  'direction1': 'N',
  'nextspace2': 'Outdoor',
  'direction2': 'E',
  'structure': 'RC',
  'length': 0.25,
  'space': 'main'},
 {'nextspace1': 'Outdoor',
  'direction1': 'N',
  'nextspace2': 'Outdoor',
  'direction2': 'E',
  'structure': 'RC',
  'length': 0.5,
  'space': 'other'},
 {'nextspace1': 'Outdoor',
  'direction1': 'N',
  'nextspace2': 'Outdoor',
  'direction2': 'E',
  'structure': 'RC',
  'length': 0.25,
  'space': 'nonliving'},
 {'nextspace1': 'Outdoor',
  'direction1': 'N',
  'nextspace2': 'Outdoor',
  'direction2': 'W',
  'structure': 'Steel',
  'length': 0.5,
  'space': 'main'},
 {'nextspace1': 'Outdoor',
  'direction1': 'N',
  'nextspace2': 'Outdoor',
  'direction2': 'W',
  'structure': 'Steel',
  'length': 1.0,
  'space': 'other'},
 {'nextspace1': 'Outdoor',
  'direction1': 'N',
  'nextspace2': 'Outdoor',
  'direction2': 'W',
  'structure': 'Steel',
  'length': 0.5,
  'space': 'nonliving'}]

### 6) 土間床等の外周部の線熱橋

- 方位、熱橋長さ[m]、線熱貫流率[W/(m・K)]などの情報は維持する。
- 「土間床等の外周部の線熱橋が属する空間種類」の情報を追加する。ただし、Lv2からLv3への変換においては、空間種類は「床下空間」のみとする。

#### Function

In [22]:
def make_earthfloorperimeter( nextspace, length, name, psi, direction = None, **kwargs ):
    
    earthfloorperimeter = {
        'nextspace': nextspace,
        'direction': direction,
        'length':    length,
        'name':      name,
        'psi':       psi,
        'space':     'underfloor',
    }
    
    if nextspace == 'Outdoor' :
        earthfloorperimeter['direction'] = direction

    return earthfloorperimeter

In [23]:
def convert_earthfloorperimeter( d ):    
    return [ make_earthfloorperimeter(**f) for f in d['EarthfloorPerimeters'] ]

#### Example

In [24]:
d = {
    'EarthfloorPerimeters': [
        { 'name': 'NW', 'nextspace': 'Outdoor', 'direction': 'NW', 'length': 2.43, 'psi': 1.8 },
        { 'name': 'NE', 'nextspace': 'Outdoor', 'direction': 'NE', 'length': 2.43, 'psi': 1.8 }
    ]
}    

convert_earthfloorperimeter(d)

[{'nextspace': 'Outdoor',
  'direction': 'NW',
  'length': 2.43,
  'name': 'NW',
  'psi': 1.8,
  'space': 'underfloor'},
 {'nextspace': 'Outdoor',
  'direction': 'NE',
  'length': 2.43,
  'name': 'NE',
  'psi': 1.8,
  'space': 'underfloor'}]

### 7) 土間床
- 面積[m<sup>2</sup>]などの情報は維持する。
- 「土間床等が属する空間種類」の情報を追加する。ただし、Lv2からLv3への変換においては、空間種類は「床下空間」のみとする。

#### Function

In [25]:
def make_earthfloor( name, area ):
    return {
        'name': name,
        'area': area,
        'space': 'underfloor'
    }
    return earthfloor

In [26]:
def convert_earthfloor( d ):        
    return [ make_earthfloor(**f) for f in d['Earthfloors'] ]

#### Example

In [27]:
d = {
    'Earthfloors': [
        { 'area': 5.0, 'name': 'other' },
        { 'area': 5.0, 'name': 'entrance' }
    ]
}    

convert_earthfloor(d)

[{'name': 'other', 'area': 5.0, 'space': 'underfloor'},
 {'name': 'entrance', 'area': 5.0, 'space': 'underfloor'}]

## 3. Lv3入力情報のコンバート（統合）
- 1)共通情報、2)開口部以外の不透明部位、3)開口部の透明部位、4)開口部の不透明部位、5) 線熱橋、6)土間床等の外周部の線熱橋、7)土間床の各コンバート関数を統合し、Lv2の入力情報をLv3の入力情報にコンバートする。

### Function

In [28]:
def convert(d):
    d_lv3_input = {}
    d_lv3_input['Common'] = convert_common(d)
    if ('Walls' in d) == True:
        d_lv3_input['Walls'] = convert_wall(d)
    if ('Windows' in d) == True:
        d_lv3_input['Windows'] = convert_window(d)
    if ('Doors' in d) == True:
        d_lv3_input['Doors'] = convert_door(d)
    if ('EarthfloorPerimeters' in d) == True:
        d_lv3_input['EarthfloorPerimeters'] = convert_earthfloorperimeter(d)
    if ('Earthfloors' in d) == True:
        d_lv3_input['Earthfloors'] = convert_earthfloor(d)
    if ('Heatbridges' in d) == True:
        d_lv3_input['Heatbridges'] = convert_heatbridge(d)

    return d_lv3_input

### Example

In [29]:
d = {
    'Common': {
        'Region': 6,
        'MainOccupantRoomFloorArea': 30.0,
        'OtherOccupantRoomFloorArea': 60.0,
        'TotalFloorArea': 120.0
    },
    'Walls': [
        { 'name': 'Ceiling', 'type': 'Ceiling', 'structure': 'wood', 'InputMethod': 'InputUA', 
          'nextspace': 'Outdoor', 'direction': 'top', 'area': 67.8, 'UA': 0.24, 'IsSunshadeInput': False },
        { 'name': 'Ceiling', 'type': 'Ceiling', 'structure': 'wood', 'InputMethod': 'InputAllDetails', 
          'nextspace': 'Outdoor', 'direction': 'top', 'IsInContactWithOutsideAir': True, 'area': 67.8,
          'Parts': [{'AreaRatio': 0.8, 'Layers': [{'name': 'wood','thick': 0.012, 'cond': 0.16, 'specH': 720 },
                                                  {'name': 'wood','thick': 0.012, 'cond': 0.16, 'specH': 720 }]},
                    {'AreaRatio': 0.2, 'Layers': [{'name': 'wood','thick': 0.012, 'cond': 0.16, 'specH': 720 },
                                                  {'name': 'wood','thick': 0.012, 'cond': 0.16, 'specH': 720 }]}],
          'IsSunshadeInput': False },
        { 'name': 'Floor', 'type': 'Floor', 'structure': 'wood', 'InputMethod': 'InputAllLayers',
          'nextspace': 'Outdoor', 'direction': 'bottom', 'IsInContactWithOutsideAir': True, 'area': 67.8,
          'ConstructionMethodAllLayers' :'FloorFrameBeamInsuljoist',
          'Parts': [{'Type': 'FloorInsulation', 'Layers': [{'name': 'wood','thick': 0.012, 'cond': 0.16, 'specH': 720 },
                                                           {'name': 'wood','thick': 0.012, 'cond': 0.16, 'specH': 720 }]},
                    {'Type': 'FloorHeatbridge', 'Layers': [{'name': 'wood','thick': 0.012, 'cond': 0.16, 'specH': 720 },
                                                           {'name': 'wood','thick': 0.012, 'cond': 0.16, 'specH': 720 }]}],
           'IsSunshadeInput': False },
        { 'name': 'Wall', 'type': 'Wall', 'structure': 'wood', 'InputMethod': 'InputAllLayers', 'nextspace': 'Outdoor', 'direction': 'N',
          'IsInContactWithOutsideAir': True, 'area': 67.8, 'ConstructionMethodAllLayers': 'WallFrameInsulcolumn',
          'Parts': [{'Type': 'WallInsulation', 'Layers': [{'name': 'wood','thick': 0.012, 'cond': 0.16, 'specH': 720 },
                                                          {'name': 'wood','thick': 0.012, 'cond': 0.16, 'specH': 720 }]},
                    {'Type': 'WallHeatbridge', 'Layers': [{'name': 'wood','thick': 0.012, 'cond': 0.16, 'specH': 720 },
                                                          {'name': 'wood','thick': 0.012, 'cond': 0.16, 'specH': 720 }]}],         
          'IsSunshadeInput': False },
        { 'name': 'Ceiling', 'type': 'Roof', 'structure': 'wood', 'InputMethod': 'InputAllLayers', 'nextspace': 'Outdoor', 'direction': 'top',
          'IsInContactWithOutsideAir': True, 'area': 67.8, 'ConstructionMethodAllLayers': 'RoofInsulRafter', 
          'Parts': [{'Type': 'RoofInsulation', 'Layers': [{'name': 'wood','thick': 0.012, 'cond': 0.16, 'specH': 720 },
                                                          {'name': 'wood','thick': 0.012, 'cond': 0.16, 'specH': 720 }]},
                    {'Type': 'RoofHeatbridge', 'Layers': [{'name': 'wood','thick': 0.012, 'cond': 0.16, 'specH': 720 },
                                                          {'name': 'wood','thick': 0.012, 'cond': 0.16, 'specH': 720 }]}],
          'IsSunshadeInput': False },
        { 'name': 'BoundaryCeiling', 'type': 'BoundaryCeiling', 'structure': 'wood', 'InputMethod': 'InputAllLayers', 'nextspace': 'Outdoor', 
          'direction': 'top', 'IsInContactWithOutsideAir': True, 'area': 67.8, 'ConstructionMethodAllLayers': 'CeilingInsulbeam', 
          'Parts': [{'Type': 'CeilingInsulation', 'Layers': [{'name': 'wood','thick': 0.012, 'cond': 0.16, 'specH': 720 },
                                                             {'name': 'wood','thick': 0.012, 'cond': 0.16, 'specH': 720 }]},
                    {'Type': 'CeilingHeatbridge', 'Layers': [{'name': 'wood','thick': 0.012, 'cond': 0.16, 'specH': 720 },
                                                             {'name': 'wood','thick': 0.012, 'cond': 0.16, 'specH': 720 }]}],
          'IsSunshadeInput': False },
        { 'name': 'Ceiling', 'type': 'Ceiling', 'structure': 'wood', 'InputMethod': 'InputUR', 'InsulationMethod': 'CeilingInsulbeam',
          'nextspace': 'Outdoor', 'direction': 'top', 'IsInContactWithOutsideAir': True, 'area': 67.8,
          'Parts': [{ 'Layers': [{'name': 'wood','thick': 0.012, 'cond': 0.16, 'specH': 720 },
                                 {'name': 'wood','thick': 0.012, 'cond': 0.16, 'specH': 720 }]}],
          'IsSunshadeInput': False },
        { 'name': 'Ceiling', 'type': 'Ceiling', 'structure': 'RC', 'InputMethod': 'InputUA', 
          'nextspace': 'Outdoor', 'direction': 'top', 'area': 67.8, 'UA': 0.24, 'IsSunshadeInput': False },
        { 'name': 'Ceiling', 'type': 'Ceiling', 'structure': 'RC', 'InputMethod': 'InputLayers', 
          'nextspace': 'Outdoor', 'direction': 'top', 'area': 67.8,
          'Parts': [{'Layers': [{'name': 'wood','thick': 0.012, 'cond': 0.16, 'specH': 720 },
                                {'name': 'wood','thick': 0.012, 'cond': 0.16, 'specH': 720 }]}],         
          'IsSunshadeInput': False },
        { 'name': 'Ceiling', 'type': 'Ceiling', 'structure': 'steel', 'InputMethod': 'InputUA', 
          'nextspace': 'Outdoor', 'direction': 'top', 'area': 67.8, 'UA': 0.24, 'IsSunshadeInput': False },
        { 'name': 'Ceiling', 'type': 'Ceiling', 'structure': 'steel', 'InputMethod': 'InputUR', 
          'nextspace': 'Outdoor', 'direction': 'top', 'IsInContactWithOutsideAir': True, 'area': 67.8, 'Radding': 0.10, 
          'Parts': [{'Layers': [{'name': 'wood','thick': 0.012, 'cond': 0.16, 'specH': 720 },
                                {'name': 'wood','thick': 0.012, 'cond': 0.16, 'specH': 720 }]}],         
          'IsSunshadeInput': False },
        { 'name': 'Ceiling', 'type': 'Ceiling', 'structure': 'other', 'InputMethod': None, 
          'nextspace': 'Outdoor', 'direction': 'top', 'area': 67.8, 'IsSunshadeInput': False }
    ],
    'Windows': [
        { 'name': 'WindowSW', 'nextspace': 'Outdoor', 'direction': 'SW', 'area': 30.25, 'UW': 6.51, 'TypeWindow': 'Single', 
          'EtaInputMethod': 'InputValue', 'TypeFrame': 'WoodOrResin', 'TypeGlass': '3WgG', 'TypeShade': 'Shoji',
          'IsSunshadeInput': True, 'Y1': 0.00, 'Y2': 1.00, 'Z': 0.60 },
        { 'name': 'WindowNW', 'nextspace': 'Outdoor', 'direction': 'NW', 'area': 3.17, 'UW': 4.65, 'TypeWindow': 'Single', 
          'EtaInputMethod': 'InputValue', 'Eta': 0.738, 'IsSunshadeInput': False },
        { 'name': 'WindowSW', 'nextspace': 'Outdoor', 'direction': 'SW', 'area': 30.25, 'UW': 6.51, 'TypeWindow': 'Double', 
          'EtaInputMethod': 'InputSpecification', 'TypeFrameInside': 'WoodOrResin', 'TypeGlassInside': '3WgG', 'TypeShadeInside': 'Shoji',
          'TypeFrameOutside': 'Steel', 'TypeGlassOutside': '3WgG', 'TypeShadeOutside': 'ExtarnalBlind',
          'IsSunshadeInput': True, 'Y1': 0.00, 'Y2': 1.00, 'Z': 0.60 },
        { 'name': 'WindowNW', 'nextspace': 'Outdoor', 'direction': 'NW', 'area': 3.17, 'UW': 4.65, 'TypeWindow': 'Double', 
          'EtaInputMethod': 'InputValue', 'EtaInside': 0.738, 'TypeGlassInside': '3WgG', 'EtaOutside': 0.738, 'TypeGlassOutside': '3WgG',
          'IsSunshadeInput': True, 'Y1': 0.00, 'Y2': 1.00, 'Z': 0.60 }
    ],
    'Doors': [
        { 'name': 'DoorNW', 'nextspace': 'Outdoor', 'direction': 'NW', 'area': 2.52, 'U': 6.51, 'IsSunshadeInput': False },
        { 'name': 'DoorNE', 'nextspace': 'Outdoor', 'direction': 'NE', 'area': 2.16, 'U': 4.65, 
          'IsSunshadeInput': True, 'Y1': 0.00, 'Y2': 1.00, 'Z': 0.60 }
    ],
    'Heatbridges': [
        { 'name': 'NE', 'structure': 'RC', 'length': 1.00, 'psi': 1.8, 
          'nextspace1': 'Outdoor', 'direction1': 'N', 'nextspace2': 'Outdoor', 'direction2': 'E' },
        { 'name': 'NW', 'structure': 'Steel', 'length': 2.00, 'psi': 1.8, 
          'nextspace1': 'Outdoor', 'direction1': 'N', 'nextspace2': 'Outdoor', 'direction2': 'W' }
    ],
    'EarthfloorPerimeters': [
        { 'name': 'NW', 'nextspace': 'Outdoor', 'direction': 'NW', 'length': 2.43, 'psi': 1.8 },
        { 'name': 'NE', 'nextspace': 'Outdoor', 'direction': 'NE', 'length': 2.43, 'psi': 1.8 }
    ],
    'Earthfloors': [
        { 'area': 5.0, 'name': 'other' },
        { 'area': 5.0, 'name': 'entrance' }
    ]
}

convert(d)

{'Common': {'Region': 6,
  'MainOccupantRoomFloorArea': 30.0,
  'OtherOccupantRoomFloorArea': 60.0,
  'TotalFloorArea': 120.0},
 'Walls': [{'name': 'Ceiling_main',
   'nextspace': 'Outdoor',
   'direction': 'top',
   'area': 16.95,
   'space': 'main',
   'type': 'Ceiling',
   'InputMethod': 'InputUA',
   'IsSunshadeInput': False,
   'UA': 0.24},
  {'name': 'Ceiling_other',
   'nextspace': 'Outdoor',
   'direction': 'top',
   'area': 33.9,
   'space': 'other',
   'type': 'Ceiling',
   'InputMethod': 'InputUA',
   'IsSunshadeInput': False,
   'UA': 0.24},
  {'name': 'Ceiling_nonliving',
   'nextspace': 'Outdoor',
   'direction': 'top',
   'area': 16.95,
   'space': 'nonliving',
   'type': 'Ceiling',
   'InputMethod': 'InputUA',
   'IsSunshadeInput': False,
   'UA': 0.24},
  {'name': 'Ceiling_main',
   'nextspace': 'Outdoor',
   'direction': 'top',
   'area': 16.95,
   'space': 'main',
   'type': 'Ceiling',
   'InputMethod': 'InputAllDetails',
   'IsSunshadeInput': False,
   'structure': 

In [30]:
d = {
    'Common':{
    'Region': 6,
        'MainOccupantRoomFloorArea': 30.0,
        'OtherOccupantRoomFloorArea': 30.0,
        'TotalFloorArea': 90.0
    },
 'Walls': [
        {'name': 'Ceiling', 'type': 'Ceiling', 'nextspace': 'Outdoor', 'direction': 'Top', 'area': 50.85, 'UA': 7.7,
         'InputMethod': 'InputUA', 'IsSunshadeInput': False},
        {'name': 'Wall_SW', 'type': 'Wall', 'nextspace': 'Outdoor', 'direction': 'SW', 'area': 30.47, 'UA': 6.67,
         'InputMethod': 'InputUA', 'IsSunshadeInput': False},
        {'name': 'Wall_NW', 'type': 'Wall', 'nextspace': 'Outdoor', 'direction': 'NW', 'area': 22.37, 'UA': 6.67,
         'InputMethod': 'InputUA', 'IsSunshadeInput': False},
        {'name': 'Wall_NE', 'type': 'Wall', 'nextspace': 'Outdoor', 'direction': 'NE', 'area': 47.92, 'UA': 6.67,
         'InputMethod': 'InputUA', 'IsSunshadeInput': False},
        {'name': 'Wall_SE', 'type': 'Wall', 'nextspace': 'Outdoor', 'direction': 'SE', 'area': 22.28, 'UA': 6.67,
         'InputMethod': 'InputUA', 'IsSunshadeInput': False},
        {'name': 'Floor', 'type': 'Floor', 'nextspace': 'Outdoor', 'direction': 'Bottom', 'area': 45.05, 'UA': 5.27,
         'InputMethod': 'InputUA', 'IsSunshadeInput': False},
        {'name': 'Floor_bath', 'type': 'Floor', 'nextspace': 'Outdoor', 'direction': 'Bottom', 'area': 3.31, 'UA': 5.27,
         'InputMethod': 'InputUA', 'IsSunshadeInput': False}
    ],
    'Windows': [
        {'name': 'WindowSW', 'nextspace': 'Outdoor', 'direction': 'SW', 'area': 22.69, 'TypeWindow': 'Single', 'EtaInputMethod': 'InputValue', 
         'etaCooling': 0.51, 'etaHeating': 0.51, 'UW': 3.49, 'IsSunshadeInput': True, 'Y1': 0, 'Y2': 1.1, 'z': 0.348},
        {'name': 'WindowNW', 'nextspace': 'Outdoor', 'direction': 'NW', 'area': 2.38, 'TypeWindow': 'Single', 'EtaInputMethod': 'InputValue',
         'etaCooling': 0.51, 'etaHeating': 0.51, 'UW': 3.49, 'IsSunshadeInput': True, 'Y1': 0, 'Y2': 1.1, 'z': 0.316},
        {'name': 'WindowNE', 'nextspace': 'Outdoor', 'direction': 'NE', 'area': 3.63, 'TypeWindow': 'Single',  'EtaInputMethod': 'InputValue',
         'etaCooling': 0.51, 'etaHeating': 0.51, 'UW': 3.49, 'IsSunshadeInput': True, 'Y1': 0, 'Y2': 1.1, 'z': 0.316},
        {'name': 'WindowSE', 'nextspace': 'Outdoor', 'direction': 'SE', 'area': 4.37, 'TypeWindow': 'Single', 'EtaInputMethod': 'InputValue',
         'etaCooling': 0.51, 'etaHeating': 0.51, 'UW': 3.49, 'IsSunshadeInput': True, 'Y1': 0, 'Y2': 1.1, 'z': 0.348}
    ],
    'Doors': [
        {'name': 'DoorNW', 'nextspace': 'Outdoor', 'direction': 'NW', 'area': 1.89, 'U': 4.65, 'IsSunshadeInput': False},
        {'name': 'DoorNE', 'nextspace': 'Outdoor', 'direction': 'NE', 'area': 1.62, 'U': 4.65, 'IsSunshadeInput': False}
    ],
    'EarthfloorPerimeters': [
        {'name': 'Entrance_NW', 'nextspace': 'Outdoor', 'direction': 'NW', 'length': 1.82, 'psi': 1.8},
        {'name': 'Entrance_NE', 'nextspace': 'Outdoor', 'direction': 'NE', 'length': 1.37, 'psi': 1.8},
        {'name': 'Entrance_floor', 'nextspace': 'OpenBackFloor', 'length': 3.19, 'psi': 1.8}
    ]
}

convert(d)

{'Common': {'Region': 6,
  'MainOccupantRoomFloorArea': 30.0,
  'OtherOccupantRoomFloorArea': 30.0,
  'TotalFloorArea': 90.0},
 'Walls': [{'name': 'Ceiling_main',
   'nextspace': 'Outdoor',
   'direction': 'Top',
   'area': 16.95,
   'space': 'main',
   'type': 'Ceiling',
   'InputMethod': 'InputUA',
   'IsSunshadeInput': False,
   'UA': 7.7},
  {'name': 'Ceiling_other',
   'nextspace': 'Outdoor',
   'direction': 'Top',
   'area': 16.95,
   'space': 'other',
   'type': 'Ceiling',
   'InputMethod': 'InputUA',
   'IsSunshadeInput': False,
   'UA': 7.7},
  {'name': 'Ceiling_nonliving',
   'nextspace': 'Outdoor',
   'direction': 'Top',
   'area': 16.95,
   'space': 'nonliving',
   'type': 'Ceiling',
   'InputMethod': 'InputUA',
   'IsSunshadeInput': False,
   'UA': 7.7},
  {'name': 'Wall_SW_main',
   'nextspace': 'Outdoor',
   'direction': 'SW',
   'area': 10.156666666666666,
   'space': 'main',
   'type': 'Wall',
   'InputMethod': 'InputUA',
   'IsSunshadeInput': False,
   'UA': 6.67},
  